In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# trunc_illuminaga_rna_data has the same illuminaga RNA expression data as that downloaded from 
# class project website, though the keys in the header of trunc_illuminaga_rna_data.tsv now match
# patient.bcr_patient_barcode from the clinical data, which gives use better mapping between the
# clinical patients dataset and the RNA dataset.
gene_exp = pd.read_table('data/trunc_combined_rna_data.tsv',
                                     header=0,
                                     index_col=0)

## Preprocessing

###  Normalize by taking log of FPKM expression values.

In [3]:
log_gene_exp_df = np.log(gene_exp.copy())

### Replace all -inf with the smallest logFPKM integral, so that we don't get weird results with the variance.

In [4]:
log_gene_exp_df.replace([np.inf, -np.inf], np.nan, inplace=True)  # must replace all -inf with NaN so that .min() will work.
min_fpkm_per_patient = list(log_gene_exp_df.min())
min_fpkm = min(min_fpkm_per_patient)
fpkm_floor = math.floor(min_fpkm)
log_gene_exp_df.replace(np.nan, fpkm_floor, inplace=True)

### Take top 10000 genes with highest variance

In [5]:
log_gene_exp_var_df = log_gene_exp_df.copy()
log_gene_exp_var_df['var'] = log_gene_exp_var_df.var(axis=1)
filtered_log_gene_exp = log_gene_exp_var_df.sort_values(by='var', ascending=False)[:10000]
filtered_log_gene_exp = filtered_log_gene_exp.ix[:, :-1]  # Remove variance column

## Formatting Data

### Get features and labels

#### NOTE: mRNA data patients are a subset of those included in COADREAD.clin.merged.txt

In [6]:
clinical_data_df = pd.read_table('data/clinical/COADREAD.clin.merged.txt', index_col=0)

In [7]:
# Don't try to print all contents of patient_dict; too big! Will freeze browser.
patient_dict = {}
patient_dict['colon'] = {}
patient_dict['rectum'] = {}

tumor_tissue_site_nan_count = 0
patient_rna_exp_barcode_nan_count = 0
patient_rna_exp_barcode_not_in_rna_dataset_count = 0

column_header_list = list(clinical_data_df.columns.values)
for column_header in column_header_list:

    tumor_tissue_site = str(clinical_data_df.loc["patient.tumor_tissue_site"][column_header])
    if tumor_tissue_site != "nan":  # We only want patients which have a label.
        
        bcr_patient_barcode = clinical_data_df.loc["patient.bcr_patient_barcode"][column_header]
        bcr_patient_barcode = bcr_patient_barcode.upper()
        
        if bcr_patient_barcode in filtered_log_gene_exp.keys():
            patient_exp_list = list(filtered_log_gene_exp[bcr_patient_barcode])
            patient_dict[tumor_tissue_site][bcr_patient_barcode] = patient_exp_list
            
    # investigating quality of my data mapping
    else:
        tumor_tissue_site_nan_count += 1

print(tumor_tissue_site_nan_count)

4


#### Balancing training set between colon and rectum tissue.

In [8]:
print(len(patient_dict['rectum']))
print(len(patient_dict['colon']))

164
454


In [9]:
rectum_dict = patient_dict['rectum']

training_patient_list = list(rectum_dict.keys())[:100]
testing_patient_list = list(rectum_dict.values())[100:]

training_feature_list = list(rectum_dict.values())[:100]
training_label_list = ['rectum' ] * len(training_feature_list)

testing_feature_list = list(rectum_dict.values())[100:]
testing_label_list = ['rectum'] * len(testing_feature_list)


colon_dict = patient_dict['colon']

training_patient_list += list(colon_dict.keys())[:100]
testing_patient_list += list(colon_dict.values())[100:]

training_feature_list += list(colon_dict.values())[:100]
training_label_list += ['colon' ] * len(list(colon_dict.values())[:100])

testing_feature_list += list(colon_dict.values())[100:]
testing_label_list += ['colon'] * len(list(colon_dict.values())[100:])

print(len(training_feature_list))
print(len(training_patient_list))

print(len(testing_feature_list))
print(len(testing_patient_list))

200
200
418
418


## Classification

In [10]:
from sklearn.svm import SVC

In [11]:
svm_classifier = SVC()
svm_classifier.fit(training_feature_list, training_label_list)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [21]:
training_feature_list[0]

[5.7628526340196951,
 7.565289323251128,
 5.8566208652370237,
 6.5215970870904201,
 6.4474890470297392,
 5.7177843229906422,
 6.2723213805164963,
 9.2640742387426513,
 -6.0,
 5.6631257442568561,
 4.9737167821762167,
 -6.0,
 -6.0,
 4.3243723626224497,
 2.0217859454416698,
 2.2731047876715373,
 -6.0,
 4.9198283594459404,
 4.6402255051513457,
 4.4453253250388229,
 3.789448509035632,
 5.8779956919776479,
 3.7134623045831057,
 0.075849312283437831,
 6.7491751703699538,
 5.2798840113707666,
 1.6853243022652333,
 1.8676396795633732,
 2.1553140275916856,
 1.4621668470317404,
 3.2204585716634107,
 4.5532145670488928,
 3.5416141820347713,
 3.7324919515568187,
 -6.0,
 2.5607868601233186,
 1.4621668470317404,
 -6.0,
 -6.0,
 3.8370774518317696,
 5.8870183355689525,
 -6.0,
 5.6210546374690651,
 1.1744924990034278,
 -6.0,
 2.7839304114375283,
 -6.0,
 2.3784622135671363,
 8.4563338883143047,
 -6.0,
 0.075849312283437831,
 -6.0,
 -6.0,
 0.31488372987692598,
 4.9200641340450577,
 4.6297545000314599,
 0.

In [12]:
training_predictions = svm_classifier.predict(training_feature_list)
testing_predictions = svm_classifier.predict(testing_feature_list)

In [13]:
# We don't want all predictions to simply be the same. We want variety in our predictions.
testing_predictions

array(['colon', 'colon', 'rectum', 'colon', 'colon', 'colon', 'rectum',
       'rectum', 'colon', 'rectum', 'colon', 'colon', 'colon', 'colon',
       'colon', 'colon', 'colon', 'colon', 'rectum', 'rectum', 'colon',
       'colon', 'rectum', 'colon', 'colon', 'rectum', 'rectum', 'colon',
       'colon', 'colon', 'rectum', 'colon', 'colon', 'rectum', 'colon',
       'colon', 'colon', 'colon', 'colon', 'rectum', 'rectum', 'rectum',
       'colon', 'rectum', 'rectum', 'colon', 'colon', 'rectum', 'colon',
       'colon', 'colon', 'colon', 'colon', 'rectum', 'colon', 'colon',
       'colon', 'rectum', 'rectum', 'rectum', 'rectum', 'colon', 'colon',
       'rectum', 'colon', 'rectum', 'colon', 'colon', 'colon', 'colon',
       'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'rectum',
       'colon', 'rectum', 'colon', 'colon', 'colon', 'colon', 'rectum',
       'rectum', 'colon', 'colon', 'colon', 'rectum', 'colon', 'colon',
       'colon', 'colon', 'colon', 'colon', 'rectum', 'rectum'

In [14]:
np.array(testing_label_list)

array(['rectum', 'rectum', 'rectum', 'rectum', 'rectum', 'rectum',
       'rectum', 'rectum', 'rectum', 'rectum', 'rectum', 'rectum',
       'rectum', 'rectum', 'rectum', 'rectum', 'rectum', 'rectum',
       'rectum', 'rectum', 'rectum', 'rectum', 'rectum', 'rectum',
       'rectum', 'rectum', 'rectum', 'rectum', 'rectum', 'rectum',
       'rectum', 'rectum', 'rectum', 'rectum', 'rectum', 'rectum',
       'rectum', 'rectum', 'rectum', 'rectum', 'rectum', 'rectum',
       'rectum', 'rectum', 'rectum', 'rectum', 'rectum', 'rectum',
       'rectum', 'rectum', 'rectum', 'rectum', 'rectum', 'rectum',
       'rectum', 'rectum', 'rectum', 'rectum', 'rectum', 'rectum',
       'rectum', 'rectum', 'rectum', 'rectum', 'colon', 'colon', 'colon',
       'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon',
       'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon',
       'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon',
       'colon', 'colon', 'colon', 'colon', 'co

### Output Classification to TSV

In [15]:
patient_list = training_patient_list + testing_patient_list
prediction_list = list(training_predictions) + list(testing_predictions)

output_tsv_df = pd.DataFrame(np.array([prediction_list]), columns=patient_list)

In [16]:
output_tsv_df.to_csv('data/both_patient_classification.tsv', sep='\t')

### Investigate Classification Statistics

In [17]:
from sklearn import metrics

In [18]:
metrics.accuracy_score(testing_label_list, testing_predictions)

0.67224880382775121

In [19]:
# Essentially returns the Recall
def get_class_accuracy(class_name, label_list, predictions):
    correct_prediction_count = 0
    class_count = 0
    for index in range(0, len(label_list)):
        if label_list[index] == class_name:
            class_count += 1
            if predictions[index] == label_list[index]:
                correct_prediction_count += 1    
    return correct_prediction_count/class_count

In [20]:
get_class_accuracy('rectum', testing_label_list, testing_predictions)

0.484375

In [21]:
get_class_accuracy('colon', testing_label_list, testing_predictions)

0.7175141242937854

In [22]:
# Essentially returns the True Prediction Rate.
def get_average_per_class_accuracy(testing_label_list, testing_predictions):
    rectum_acc = get_class_accuracy('rectum', testing_label_list, testing_predictions)
    colon_acc = get_class_accuracy('colon', testing_label_list, testing_predictions)
    return (rectum_acc + colon_acc) / 2

In [23]:
get_average_per_class_accuracy(testing_label_list, testing_predictions)

0.6009445621468927

In [53]:
def get_recall(class_name, y_true, y_pred):
    bin_y_true = [1 if label == class_name else 0 for label in y_true]
    bin_y_pred = [1 if label == class_name else 0 for label in y_pred]
    return metrics.recall_score(bin_y_true, bin_y_pred)

In [54]:
rectum_true_prediction_rate = get_recall('rectum', testing_label_list, testing_predictions)
print(rectum_true_prediction_rate)

0.484375


In [55]:
colon_true_prediction_rate = get_recall('colon', testing_label_list, testing_predictions)
print(colon_true_prediction_rate)

0.717514124294


In [56]:
def get_precision(class_name, y_true, y_pred):
    bin_y_true = [1 if label == class_name else 0 for label in y_true]
    bin_y_pred = [1 if label == class_name else 0 for label in y_pred]
    return metrics.precision_score(bin_y_true, bin_y_pred)

In [60]:
rectum_precision = get_precision('rectum', testing_label_list, testing_predictions)
print(rectum_precision)

0.236641221374


In [61]:
colon_precision = get_precision('colon', testing_label_list, testing_predictions)
print(colon_precision)

0.885017421603


In [59]:
# Balanced Error Rate (BER)
rectum_false_prediction_rate = 1 - rectum_true_prediction_rate
colon_false_prediction_rate = 1 - colon_true_prediction_rate
print(0.5 * (rectum_false_prediction_rate + colon_false_prediction_rate))

0.399055437853


In [ ]:
### BER of 0.5 would for coin flipping classifier; 0.399 means that our classifier is a bit better.

In [69]:
# Harmonic mean
rectum_F1_score = 2 * (rectum_precision*rectum_true_prediction_rate)/(rectum_precision+rectum_true_prediction_rate)
print(rectum_F1_score)

0.317948717949


In [71]:
# Harmonic mean
colon_F1_score = 2 * (colon_precision*colon_true_prediction_rate)/(colon_precision+colon_true_prediction_rate)
print(colon_F1_score)

0.792511700468
